## Import Library

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re

In [ ]:
#import stopword
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
#tokenize
from nltk.tokenize import TweetTokenizer

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data

In [ ]:
# Load dataset
def load_data():
    df = pd.read_csv('/content/drive/MyDrive/dataset_depresi.csv')
    return df

df = load_data()
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(df[['Label', 'Tweet']])
df.columns = (['label', 'text'])
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


label  \
0    1       
1    0       
2    1       
3    1       
4    1       
..  ..       
154  0       
155  0       
156  0       
157  1       
158  1       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   159 non-null    int64 
 1   text    159 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.6+ KB


## Data Cleaning

In [ ]:
#remove user
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)
  return input_txt
df['remove_user'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")
df

label  \
0    1       
1    0       
2    1       
3    1       
4    1       
..  ..       
154  0       
155  0       
156  0       
157  1       
158  1       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
def remove(tweet):
  #remove angka
  tweet = re.sub('[0-9]+', '', tweet)
  # remove stock market tickers like $GE
  tweet = re.sub(r'\$\w*', '', tweet)
  # remove old style retweet text "RT"
  tweet = re.sub(r'^RT[\s]+', '', tweet)
  # remove hashtags
  # only removing the hash # sign from the word
  tweet = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', ' ', tweet)
  return tweet
df['remove_user'] = df['remove_user'].apply(lambda x: remove(x))
#df.sort_values("remove_http", inplace = True)
#df.drop_duplicates(subset ="remove_http", keep = 'first', inplace = True)
df

label  \
0    1       
1    0       
2    1       
3    1       
4    1       
..  ..       
154  0       
155  0       
156  0       
157  1       
158  1       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## stopword, punctuation, stemming

In [ ]:
# Happy Emoticons
emoticons_happy = set([
':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
'=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
'<3'
])
# Sad Emoticons
emoticons_sad = set([
':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
':c', ':{', '>:\\', ';('
])
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
def clean_tweets(tweet):
  # remove stock market tickers like $GE
  tweet = re.sub(r'\$\w*', '', tweet)
  # remove old style retweet text "RT"
  tweet = re.sub(r'^RT[\s]+', '', tweet)
  # remove hyperlinks
  tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
  # remove hashtags
  # only removing the hash # sign from the word
  tweet = re.sub(r'#', '', tweet)
  #remove coma
  tweet = re.sub(r',','',tweet)
  #remove angka
  tweet = re.sub('[0-9]+', '', tweet)
  # tokenize tweets
  tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
  tweet_tokens = tokenizer.tokenize(tweet)
  tweets_clean = []
  for word in tweet_tokens:
    if (word not in stopwords_indonesia and # remove stopwords
       word not in emoticons and # remove emoticons 
       word not in string.punctuation): # remove punctuation
       #tweets_clean.append(word)
       #stem_word = stemmer.stem(word) # stemming word
       tweets_clean.append(word)
  return tweets_clean
df['tweet_clean'] = df['remove_user'].apply(lambda x: clean_tweets(x))
df

label  \
0    1       
1    0       
2    1       
3    1       
4    1       
..  ..       
154  0       
155  0       
156  0       
157  1       
158  1       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
#remove punct
def remove_punct(text):
  text = " ".join([char for char in text if char not in string.punctuation])
  return text
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))
df

label  \
0    1       
1    0       
2    1       
3    1       
4    1       
..  ..       
154  0       
155  0       
156  0       
157  1       
158  1       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
# Replace word
slang_dict =  pd.read_csv('/content/drive/MyDrive/kamusreplace.csv', encoding='latin-1', header=None)
slang_dict = slang_dict.rename(columns={0: 'original', 
                                        1: 'replacement'})
slang_dict = slang_dict.astype(str)
slang_dict

,original,replacement
0,abis,habis
1,ad,ada
2,adlh,adalah
3,afaik,sejauh yang saya tahu
4,ahaha,haha
...,...,...
2407,that,itu
2408,is,adalah
2409,nggabisa,tidak bisa
2410,telkoms,telkomsel


In [ ]:
slang_dict_map = dict(zip(slang_dict['original'], slang_dict['replacement']))

def normalize_slang(text):
    return ' '.join([slang_dict_map[word] if word in slang_dict_map else word for word in text.split(' ')])
df['replace_slang_word'] = df['Tweet'].apply(lambda x: normalize_slang(x))
df

label  \
0    1       
1    0       
2    1       
3    1       
4    1       
..  ..       
154  0       
155  0       
156  0       
157  1       
158  1       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['clean_tweet'] = df['replace_slang_word'].apply(remove_singl_char)
df

label  \
0    1       
1    0       
2    1       
3    1       
4    1       
..  ..       
154  0       
155  0       
156  0       
157  1       
158  1       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
df['clean_tweet'] = df['clean_tweet'].apply(remove_emojis)
df

label  \
0    1       
1    0       
2    1       
3    1       
4    1       
..  ..       
154  0       
155  0       
156  0       
157  1       
158  1       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
from string import ascii_letters
df['clean_tweet'] = \
    df['clean_tweet'].map(lambda x: x.lstrip('+-').rstrip(ascii_letters))

In [ ]:
def rep_char(str1):
    tchr = str1.group(0)
    if len(tchr) > 1:
        return tchr[0:1] # can change the value here on repetition
def unique_char(rep, sent_text):
    convert = re.sub(r'(\w)\1+', rep, sent_text) 
    return convert
df['clean_tweet']=df['clean_tweet'].apply(lambda x : unique_char(rep_char,x))

In [ ]:
df['clean_tweet'] = df['clean_tweet'].str.replace('—','')

In [ ]:
df['clean_tweet'] = df['clean_tweet'].str.replace(".","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
df['remove_user'].iloc[49]

'On board PPD- Kor  Sejak ga ada PLB di Bis, ga ada yg tegur    ceritanya touring.. "CERITANYA" :v (with you) [vid] —   ceritanya di PWT (with you at ) —   ` Selamat natal (with Frans, Imanuell, and  others at Ta Wan Cipinang Indah Mall) —   hello twitter  iya pak hahaha Feel the bass!! #Harman #Kardon #HK #HarmanKardon #Onyx #Speaker #Sades… (at MT. Haryono (MTH) Square) [pic] —   SIM (at ) —   #Pathdaily [pic] —   Pulang (at TM -) —   Brief (at ) [pic] —   dingin euy (at Zodiak Hotel Kebon Kawung) —   🍴 (at ) [pic] —     Live, full… (w/ you at MT. Haryono (MTH) Square) —   #Pathdaily [pic] —   😂😂😂😂😂😂😂 (at MT. Haryono (MTH) Square) [pic] —   mainnya kurang jauh? (with you at ) [pic] —   deepwater horizon (with you at ) —   #pathdaily (with you at MT. Haryono (MTH) Square) [pic] —   fine 😊 (at MT Haryono Square) [pic] —   hmmmmm leh ugha (at MT. Haryono (MTH) Square) [pic] —   Home sweet home (with you at MT. Haryono (MTH) Square) —   nunggu bis serasa nunggu jodoh :v lama coeg ♫ The M

In [ ]:
df['Tweet'].iloc[49]

"on board ppd kor ga plb bis ga yg tegur ceritanya touring .. ceritanya v with you vid — ceritanya pwt with you at — selamat natal with frans imanuell and others at ta wan cipinang indah mall — hello twitter iya hahaha feel the bass harman kardon hk harmankardon onyx speaker sades … at mt haryono mth square pic — sim at — pathdaily pic — pulang at tm — brief at pic — dingin euy at zodiak hotel kebon kawung — 🍴 at pic — live full … w you at mt haryono mth square — pathdaily pic — 😂 😂 😂 at mt haryono mth square pic — mainnya with you at pic — deepwater horizon with you at — pathdaily with you at mt haryono mth square pic — fine 😊 at mt haryono square pic — hmmm leh ugha at mt haryono mth square pic — home sweet home with you at mt haryono mth square — nunggu bis serasa nunggu jodoh v coeg ♫ the man who can't be moved by at — ribet ripet nya .. ss aja v simple with era risna at pic — hmmm ♫ monsters feat katie sky by timeflies with era risna at — meet up with athian at jl.braga pic — maka

In [ ]:
df['clean_tweet'].iloc[49]

"on board pd kor tidak plb bis tidak yang tegur ceritanya touring  ceritanyawith you vidceritanya pwt with you atselamat natal with frans imanuel and others at ta wan cipinang indah malhelo twiter iya hahaha fel the bas harman kardon hk harmankardon onyx speaker sades  at makan teman haryono mth square picsim atpathdaily picpulang at tmbrief at picdingin euy at zodiak hotel kebon kawung at piclive ful  saya you at makan teman haryono mth squarepathdaily pic at makan teman haryono mth square picmainya with you at picdepwater horizon with you atpathdaily with you at makan teman haryono mth square picfineat makan teman haryono square pichm boleh ugha at makan teman haryono mth square pichome swet home with you at makan teman haryono mth squaretungu bis serasa tungu jodohjancukthe man who can' be moved by atribet ripet nya  s sajasimple with era risna at pichmmonsters feat katie sky by timeflies with era risna atmet up with athian at jlbraga picmakan ber berasa ber total kelatepost picpath

In [ ]:
df.to_csv(r'/content/drive/MyDrive/full_clean.csv', index = False, header=True)